In [ ]:
#!pip install -U qiskit
#!pip install -U qiskit_nature
#!pip install -U qiskit_aer
#!pip install -U qiskit_algorithms

In [ ]:
%cd ..

In [ ]:
mol = 'bh3.txt'

In [ ]:
### Qiskit Nature
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit.quantum_info import SparsePauliOp


from qiskit_nature.second_q.mappers import BravyiKitaevMapper,BravyiKitaevSuperFastMapper,JordanWignerMapper,ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC, UCCSD
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, ExcitedStatesEigensolver, QEOM, EvaluationRule

### Qiskit Runtime
from qiskit.primitives import BackendEstimator as Estimator
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator, Aer
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService


### Qiskit Algorithms
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, SLSQP,COBYLA,SPSA
from qiskit_algorithms import NumPyMinimumEigensolver


### Other
import copy
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Settings
###################################
basis = 'sto-3g'
unit = DistanceUnit.ANGSTROM
qubit_reduction = False
map_type = "BravyiKitaev"
shots = 50000
optimizer = SLSQP()
ini_params = [ 0, 0, 0]
dx = 1e-05  # displacement for numerical gradient
###################################

In [ ]:
class Molecule:
    def __init__(self):
        self.atoms = []
        self.energy = None
        self.gradient = None

    def add_atom(self, atom_type: str, x: float, y: float, z: float):
        atom = {
            "type": atom_type,
            "coordinates": (x, y, z)
        }
        self.atoms.append(atom)

    def load_from_file(self, filename: str):
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Reading the atoms section
        for line in lines[3:]:  # Skip the first three lines
            if line.strip() == "*":  # End
                break
            parts = line.split()
            atom_type = parts[0]
            x, y, z = map(float, parts[1:4])
            self.add_atom(atom_type, x, y, z)

    def get_pyscf_driver(self, unit: DistanceUnit = DistanceUnit.ANGSTROM, basis: str = "sto-3g"):
        # Create molecular geometry string
        geometry_str = "\n".join(f"{atom['type']} {atom['coordinates'][0]} {atom['coordinates'][1]} {atom['coordinates'][2]}"
                                 for atom in self.atoms)
        
        # Generate and return the PySCFDriver
        driver = PySCFDriver(atom=geometry_str, unit=unit, basis=basis)
        
        problem = driver.run()
        
        return problem
    
    def displace_atom(self, index: int, dx: tuple):
        if index < 0 or index >= len(self.atoms):
            raise IndexError("Atom index is out of range.")
            
        # Add displacement dx to the atom's coordinates
        self.atoms[index]["coordinates"] = [ self.atoms[index]["coordinates"][i] + dx[i] for i in range(3)]
    
    def displace_atom_coordinate(self, atom_index: int, axis: int, dx: float):
        if atom_index < 0 or atom_index >= len(self.atoms):
            raise IndexError("Atom index is out of range.")
        if axis not in (0, 1, 2):
            raise ValueError("Axis must be 0 (x), 1 (y), or 2 (z).")

        # Update the specified coordinate - change
        self.atoms[atom_index]["coordinates"][axis] += dx
        
    def __repr__(self):
        atom_repr = "\n".join(f"{atom['type']}: {atom['coordinates']}" for atom in self.atoms)
        return f"Energy: {self.energy}\nGradient: {self.gradient}\nAtoms:\n{atom_repr}"


In [ ]:
### Notebook - Molecule structure testing
#molecule = Molecule()
#molecule.load_from_file("example2.txt")
#problem = molecule.get_pyscf_driver(unit=unit,basis=basis)
#molecule.displace_atom(2, (0.1, -0.2, 0.3))
#molecule.displace_atom_coordinate(2, 0, 0.1)
#print(molecule)
#print(problem)
#second_q_ops = problem.second_q_ops()
#main_op = second_q_ops[0]
#print(main_op)

In [ ]:
def compute_energy_and_grad(mol,dx, shots, qubit_reduction, mapper_type, parameters=None, opt=SLSQP()):


    # Molecule displacement setup
    #############################################################################
    
    # r
    molecule = mol
    problem = molecule.get_pyscf_driver(unit=unit,basis=basis)
    e_nr = problem.nuclear_repulsion_energy
    main_op = problem.hamiltonian.second_q_op()
    
    
    # r - dx
    molecule_m = copy.deepcopy(mol)
    # change - dont know the displacement r - dx
    molecule_m.displace_atom(1, (0, 0, -dx))
    #molecule_m.displace_atom_coordinate(1, 2, -dx)
    problem_mdx = molecule_m.get_pyscf_driver(unit=unit,basis=basis)
    e_nr_mdx = problem_mdx.nuclear_repulsion_energy
    second_q_ops_mdx = problem_mdx.second_q_ops()
    main_op_mdx = problem_mdx.hamiltonian.second_q_op()

    # r + dx
    molecule_p = copy.deepcopy(mol)
    # change - dont know the displacement r + dx
    molecule_p.displace_atom(1, (0, 0, dx))
    #molecule_p.displace_atom_coordinate(1, 2, dx)
    
    problem_pdx = molecule_p.get_pyscf_driver(unit=unit,basis=basis)
    e_nr_pdx = problem_pdx.nuclear_repulsion_energy
    second_q_ops_pdx = problem_pdx.second_q_ops()
    main_op_pdx = problem_pdx.hamiltonian.second_q_op()
    
    
    num_particles = (problem.num_alpha,
                    problem.num_beta)

    num_spatial_orbitals = problem.num_spatial_orbitals
    num_spin_orbitals = 2 * problem.num_spatial_orbitals
    #############################################################################

    
    #############################################################################
    mapper = None
    if mapper_type == 'JordanWigner':
        mapper = JordanWignerMapper()
    elif mapper_type == 'ParityMapper':
        if qubit_reduction == False:
            mapper = ParityMapper(num_particles=problem.num_particles)
        else:
            mapper = ParityMapper(num_particles=problem.num_particles,two_qubit_reduction=True)
    elif mapper_type == 'BravyiKitaev':
        mapper = BravyiKitaevMapper()
    elif mappper_type == 'Superfast':
        mapper = BravyiKitaevSuperFastMapper(num_particles=problem.num_particles)

    # JordanWigner - https://web.archive.org/web/20191103083720/http://michaelnielsen.org/blog/archive/notes/fermions_and_jordan_wigner.pdf
    # Parity - https://arxiv.org/pdf/1701.08213
    # BravyiKitaev - https://www.sciencedirect.com/science/article/abs/pii/S0003491602962548
    # Superfast - https://arxiv.org/pdf/1712.00446

    
    # map to qubit operators
    qubit_op = mapper.map(main_op)
    qubit_op_mdx = mapper.map(main_op_mdx)
    qubit_op_pdx = mapper.map(main_op_pdx)

    #############################################################################

    # Classical - Exact solution with NUMPY
    #############################################################################
    numpy_solver = NumPyMinimumEigensolver()
    ret_exact = numpy_solver.compute_minimum_eigenvalue(qubit_op)
    ret_exact_mdx = numpy_solver.compute_minimum_eigenvalue(qubit_op_mdx)
    ret_exact_pdx = numpy_solver.compute_minimum_eigenvalue(qubit_op_pdx)
    e_fci = ret_exact._eigenvalue.real + e_nr
    e_fci_mdx = ret_exact_mdx.eigenvalue.real + e_nr_mdx
    e_fci_pdx = ret_exact_pdx.eigenvalue.real + e_nr_pdx
    grad_fci = (e_fci_pdx - e_fci_mdx) / 2 / dx
    #############################################################################

    # PREPARE QUANTUM CIRCUIT
    #############################################################################
    
    #######
    # Simulator
    aer_sim = AerSimulator()
    
    aer_sim.set_options(
        max_parallel_threads = 16,
        max_parallel_experiments = 16,
        max_parallel_shots = 1,
        statevector_parallel_threshold = 16
    )
    
    estimator = Estimator(backend=aer_sim)
    ######

    # QURI functionality - maybe overwrite
    # https://docs.quantum.ibm.com/guides/qunasys-quri-chemistry#quri-chemistry-with-uccsd
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    
    #ansatz = UCC(num_spatial_orbitals=num_spatial_orbitals, num_particles=num_particles, qubit_mapper=mapper,
    #           excitations='d')

    vqe_solver = VQE(estimator, ansatz, SPSA(maxiter=100))
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters
    #############################################################################

    solver = GroundStateEigensolver(mapper, vqe_solver)
    result_cs = solver.solve(problem, aux_operators={'qubit_op_mdx': qubit_op_mdx, 'qubit_op_pdx': qubit_op_pdx})
    print(result_cs)

    # COMPUTE - QUANTUM
    #############################################################################
    e = result_cs.eigenvalues + e_nr

    e_pdx = float(result_cs.aux_operators_evaluated[0].get('qubit_op_pdx')) + e_nr_pdx
    e_mdx = float(result_cs.aux_operators_evaluated[0].get('qubit_op_mdx')) + e_nr_mdx
    
    grad_cs = ((e_pdx - e_mdx) / (2*dx))
    
    #############################################################################

    print("Nuclear repulsion energy:", e_nr)
    print("Electronic groundstate energy:", result_cs.groundenergy)
    print("Overall Energy: ", e)
    print("Energy in +:", e_pdx)
    print("Energy in -:", e_mdx)
    print("Gradient (CS):", grad_cs)
    #print("Gradient (brute):", grad_brute)
    print("Exact energy:", e_fci)
    print("Exact energy in +:", e_fci_pdx)
    print("Exact energy in -:", e_fci_mdx)
    print("Gradient (FCI):", grad_fci)

    # Energy, energy + dx, energy - dx, gradient correlated_sampling # parameters of VQE
    return e, e_pdx, e_mdx, grad_cs, result_cs


In [ ]:
molecule = Molecule()
molecule.load_from_file(mol)
print(molecule)

In [ ]:
molecule = Molecule()
molecule.load_from_file(mol)

energy, energy_p, energy_m, gradient, parameters = compute_energy_and_grad(molecule ,dx,\
                                                                           shots, qubit_reduction, \
                                                                           map_type, parameters=None, \
                                                                           opt=SLSQP())



####output to QCxMS


In [ ]:
problem = molecule.get_pyscf_driver(unit=unit,basis=basis)
e_nr = problem.nuclear_repulsion_energy
main_op = problem.hamiltonian.second_q_op()


In [ ]:
e_nr = problem.nuclear_repulsion_energy

In [ ]:
num_particles = (problem.num_alpha,
                problem.num_beta)

num_spatial_orbitals = problem.num_spatial_orbitals
num_spin_orbitals = 2 * problem.num_spatial_orbitals
(num_particles, num_spatial_orbitals)

In [ ]:
# fails
# mapper = BravyiKitaevMapper()
mapper = ParityMapper(num_particles=problem.num_particles)
qubit_op = mapper.map(main_op)

In [ ]:
numpy_solver = NumPyMinimumEigensolver()
ret_exact = numpy_solver.compute_minimum_eigenvalue(qubit_op)


In [ ]:
ret_exact._eigenvalue.real + e_nr

In [ ]:
aer_sim = AerSimulator()

'''
aer_sim.set_options(
    max_parallel_threads = 16,
    max_parallel_experiments = 16,
    max_parallel_shots = 1,
    statevector_parallel_threshold = 16
)
'''

estimator = Estimator(backend=aer_sim)
######

# QURI functionality - maybe overwrite
# https://docs.quantum.ibm.com/guides/qunasys-quri-chemistry#quri-chemistry-with-uccsd
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

#ansatz = UCC(num_spatial_orbitals=num_spatial_orbitals, num_particles=num_particles, qubit_mapper=mapper,
#           excitations='d')

vqe_solver = VQE(estimator, ansatz, SPSA(maxiter=100))
vqe_solver.initial_point = [0.0] * ansatz.num_parameters
#############################################################################

solver = GroundStateEigensolver(mapper, vqe_solver)

In [ ]:
result_cs = solver.solve(problem)